In [1]:
import pandas as pd
import os
import numpy as np
import geojson
import cv2
from json.decoder import JSONDecodeError


In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
PATH_hovernet = '/media/user/Seagate_RNA/PCNSL_PSL/3_hovernet_normalisation_reinhard_224/'
PATH_output = '/media/user/Seagate_RNA/PCNSL_PSL/4_df_count_cells/'

In [1]:
patients = os.listdir(PATH_hovernet)
print(len(patients))

In [5]:
columns_df = {'patient' : [], 'patch' : [], 'cells_tot' : [], 'cells_filtre_bords' : [], 'cells_filtre_diametre' : [], 'cells_filtre_tot' : []}
df = pd.DataFrame.from_dict(columns_df)
df

,patient,patch,cells_tot,cells_filtre_bords,cells_filtre_diametre,cells_filtre_tot


In [6]:
#removing nuclei located at the border and nuclei which don't have a number of pixels  in the range [32:350] 
width = 224
marge = 1
res = 0.23

nbre_pix_inf = 32
nbre_pix_sup = 350

In [7]:
from PIL import Image, ImageDraw
def function_cell(cell) :
    width = 224
    poly_cell = []
    for x in cell["contour"] :
        poly_cell.append((x[0], x[1]))
    width = width
    height = width
    
    mask_cell = np.zeros((width, width), dtype = int)
    
    img = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img).polygon(poly_cell, outline = 1, fill=1)
    mask = np.array(img)
    mask_cell += mask
    
    mask_cell = np.array(mask_cell, dtype = 'uint8')

    
    return mask_cell

In [8]:
def nbre_pixels(cnt1):
    mask_cell = function_cell(cnt1)
    number_of_white_pix = np.sum(mask_cell != 0)
    return number_of_white_pix

In [11]:
for patient in patients :
    if patient not in [''] :
        if not os.path.exists(PATH_output+patient+'_.csv'):# and patient not in ['19B2703-V']:
            print(patient)
            PATH_json = PATH_hovernet+patient+'/json/' 
            columns_df = {'patient' : [], 'patch' : [], 'cells_tot' : [], 'cells_filtre_bords' : [], 'cells_filtre_diametre' : [], 'cells_filtre_tot' : []}
            df = pd.DataFrame.from_dict(columns_df)
            jsons = os.listdir(PATH_json)
            print(len(jsons))
            for patch in jsons : 
                if patch not in ['']:#liste:
                    #print(patch)
                    with open(PATH_json+patch) as f :
                        nuclei = geojson.load(f)


                    key_keep = []
                    key_not_keep_bords = []
                    key_not_keep_diametre = []
                    key_not_keep = []

                    cells_tot = len(list(nuclei['nuc'].keys()))
                    
                    for key in list(nuclei['nuc'].keys()) :
                        for x in nuclei['nuc'][key]['contour'] :
                            if x[0] < marge or x[0] > (width-marge) or x[1] < marge or x[1] > (width-marge) :
                                key_not_keep_bords.append(key)     
                            pix_count = nbre_pixels(nuclei['nuc'][key])

                            if pix_count < nbre_pix_inf or pix_count > nbre_pix_sup : #
                                key_not_keep_diametre.append(key)

                            key_not_keep_bords = list(set(key_not_keep_bords))
                            key_not_keep_diametre = list(set(key_not_keep_diametre))
                            key_not_keep = key_not_keep_bords + key_not_keep_diametre
                            key_not_keep = list(set(key_not_keep))

                        for x in nuclei['nuc'][key]['contour'] :
                            if key not in key_not_keep  :
                                key_keep.append(key) 
                    print("bzz", len(key_keep))

                    new_row = {'patient' : patient, 'patch' : patch, 'cells_tot' : cells_tot, 'cells_filtre_bords' : cells_tot - len(key_not_keep_bords), 'cells_filtre_diametre' : cells_tot - len(key_not_keep_diametre), 'cells_filtre_tot' : cells_tot - len(key_not_keep)}
                    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

                #print(len(df))
            df = df.sort_values(by=['cells_filtre_tot'], ascending = False)
            df.to_csv(PATH_output+patient+'.csv')
        else:
            print(patient, 'déjà traité')

N0400068
3912
bzz 0
bzz 0
bzz 63
bzz 62
bzz 0
bzz 41
bzz 16
bzz 0
bzz 0
bzz 0
bzz 40
bzz 0
bzz 0
bzz 0
bzz 0
bzz 20
bzz 0
bzz 0
bzz 0
bzz 0
bzz 27
bzz 21
bzz 0
bzz 91
bzz 72
bzz 116
bzz 346
bzz 93
bzz 79
bzz 115
bzz 92
bzz 30
bzz 0
bzz 19
bzz 20
bzz 14
bzz 0
bzz 0
bzz 28
bzz 161
bzz 156
bzz 0
bzz 57
bzz 46
bzz 245
bzz 122
bzz 17
bzz 86
bzz 84
bzz 19
bzz 38
bzz 30
bzz 88
bzz 16
bzz 0
bzz 0
bzz 0
bzz 0
bzz 30
bzz 0
bzz 0
bzz 0
bzz 0
bzz 0
bzz 0
bzz 0
bzz 0
bzz 43
bzz 74
bzz 112
bzz 54
bzz 24
bzz 53
bzz 17
bzz 0
bzz 47
bzz 0
bzz 0
bzz 0
bzz 21
bzz 0
bzz 20
bzz 20
bzz 141
bzz 12
bzz 56
bzz 109
bzz 60
bzz 31
bzz 51
bzz 60
bzz 64
bzz 50
bzz 60
bzz 278
bzz 184
bzz 34
bzz 67
bzz 0
bzz 0
bzz 10
bzz 67
bzz 29
bzz 0
bzz 99
bzz 15
bzz 17
bzz 25
bzz 87
bzz 0
bzz 0
bzz 14
bzz 20
bzz 13
bzz 0
bzz 34
bzz 20
bzz 29
bzz 20
bzz 0
bzz 0
bzz 0
bzz 0
bzz 0
bzz 0
bzz 0
bzz 19
bzz 37
bzz 49
bzz 30
bzz 32
bzz 0
bzz 0
bzz 10
bzz 0
bzz 228
bzz 142
bzz 23
bzz 0
bzz 0
bzz 14
bzz 17
bzz 17
bzz 45
bzz 0
bzz 33
bzz 3

KeyboardInterrupt: 

# To have some statistics

In [ ]:
print(df_tot['cells_tot_mean'].mean())

In [ ]:
df_tot['cells_filtre_bords_mean'].mean()
print(df_tot['cells_filtre_bords_mean'].mean()*100/df_tot['cells_tot_mean'].mean())

In [ ]:
df_tot['cells_filtre_diametre_mean'].mean()
print(df_tot['cells_filtre_diametre_mean'].mean()*100/df_tot['cells_tot_mean'].mean())

In [ ]:
df_tot['cells_filtre_tot_mean'].mean()
print(df_tot['cells_filtre_tot_mean'].mean()*100/df_tot['cells_tot_mean'].mean())